In [1]:
from dotenv import load_dotenv
import os
import re
import os
from groq import Groq

In [ ]:
# from stockfish import Stockfish
# from llama_cpp import Llama

# # Set paths
# # Initialize Stockfish
# stockfish = Stockfish(STOCKFISH_PATH)
# stockfish.set_skill_level(20)  # Max strength

# # Initialize Llama model
# llm = Llama(model_path=LLAMA_MODEL_PATH)

# def get_best_move(fen):
#     """Get the best move from Stockfish."""
#     stockfish.set_fen_position(fen)
#     return stockfish.get_best_move()

# def explain_move(fen):
#     """Use Llama to explain the move."""
#     move = get_best_move(fen)
#     #prompt = f"You are a chess coach. Explain why '{move}' is a best move in this position: {fen}.You have to tell the children everything or else they would never understand the things clearly , try to be a bit jolly."
#     prompt=f"Hello,World"
#     output = llm(prompt, max_tokens=100, stop=["\n"])
#     return move, output["choices"][0]["text"]

# # Example usage
# fen = "r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/2N2N2/PPPP1PPP/R1BQ1RK1 b kq - 7 5"  # Example FEN position
# move, explanation = explain_move(fen)

# print(f"Best Move: {move}")
# print(f"Explanation: {explanation}")

In [ ]:
load_dotenv()  # Load variables from .env
api_key = os.environ.get("GROQ_API_KEY")
STOCKFISH_PATH=os.environ.get("STOCKFISH_PATH")
LLAMA_MODEL_PATH=os.environ.get("LLAMA_MODEL_PATH")
print(api_key)  # Verify if it's loaded correctly

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


In [37]:
def clean(response):
    response=re.sub(r'<think>.*?</think>','',response,flags=re.DOTALL)
    return response

In [ ]:
client = Groq(
    # This is the default and can be omitted
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a chess coach , explain it well as if you thought of it"
        },
        {
            "role": "user",
            "content": "Justify why casteling e8g8 is the best movie for the FEN r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/2N2N2/PPPP1PPP/R1BQ1RK1 b kq - 7 5",
        }
    ],
    model="deepseek-r1-distill-llama-70b",
)

print(clean(chat_completion.choices[0].message.content))



The best move for Black in the given position is to castle kingside, moving the king from e8 to g8 and the rook from h8 to f8. This move, denoted as O-O, is optimal for several strategic reasons:

1. **King Safety]:** The king is currently on e8, which is somewhat exposed with an empty e7 square. Castling moves the king to g8, placing it behind the pawns on f7 and g7, enhancing its protection.

2. **Rook Activation:** After castling, the rook moves to f8, connecting both rooks and enabling them to work together in defending the position or supporting potential attacks.

3. **Prophylactic Measure:** This move is a preventive strategy to prevent potential threats from White's pieces, such as the bishop on b5 and the pawn on d5, which could otherwise target the king.

4. **Flexibility:** Castling kingside does not commit Black to a specific pawn structure, allowing flexibility in responding to future developments by White.

By castling kingside, Black secures their king, connects their 